# LLMs for Machine Learning

https://www.anaconda.com/products/ai-navigator

`OpenHermes-2.5-Mistral-7B_Q4_K_M.gguf`

In [1]:
import requests

url = "http://localhost:8080/chat/completions"
headers = {"Content-Type": "application/json"}
data = {
    "model": "model",
    "messages": [
        {"role": "system", "content": "Always answer in rhymes. Use at most 5 sentences"},
        {"role": "user", "content": "What are large language models?"}
    ],
    "temperature": 0.7,
}

response = requests.post(url, json=data, headers=headers)

print(response.json()['choices'][0]['message']['content'])


Large language models are complex, AI-powered tools,
That learn and process words in ways that seem like rules,
They can analyze, generate, and comprehend with ease,
A marvel of technology that helps us with our needs.


## Labeling Data

In [2]:
data = {
    "model": "model",
    "messages": [
        {"role": "system", "content": "Do a sentiment analysis. Only answer positive, negative or neutral as a response"},
        {"role": "user", "content": "It was a waste of time"}
    ],
    "temperature": 0.7,
}

response = requests.post(url, json=data, headers=headers)

print(response.json()['choices'][0]['message']['content'])


Negative


## Anomaly Detection

In [10]:
data = """
date         value
2024-02-01   10
2024-02-02   12
2024-02-03   11
2024-02-04   20
2024-02-05   13
2024-02-06   14
2024-02-07   13
2024-03-08   14
"""

data = {
    "model": "model",
    "messages": [
        {"role": "system", "content": "Do an anomaly detection. If there is an anomaly only report the date, otherwise say None"},
        {"role": "user", "content": data}
    ],
    "temperature": 0.7,
}

response = requests.post(url, json=data, headers=headers)

print(response.json()['choices'][0]['message']['content'])

Anomaly detected on 2024-02-04


# DSPY

https://dspy.ai/

`DSPy is the framework for programming—rather than prompting—language models`

In [11]:
# !pip install dspy 

In [12]:
import dspy
import importlib
version = importlib.metadata.version('dspy')

print(version)

2.5.43


In [13]:
# OpenHermes-2.5-Mistral-7B

lm = dspy.LM('openai/model', api_key='YOUR_OPENAI_API_KEY', base_url='http://127.0.0.1:8080', cache=False)
dspy.configure(lm=lm)


In [14]:
# lm('hi')

In [15]:
lm.inspect_history()

In [16]:
lm.history

[]

In [17]:
result = lm(
    messages=[
        {
            'role': 'system',
            'content': 'You are a famous poet. You always answer using poems.'
        },
        {
            'role': 'user',
            'content': 'What are Large language models?'
        }
    ],
    temperature=0.2
)

print(result[0])

In the realm of artificial intelligence,
A creature of code and great might,
Large language models emerge,
To shape words and thought with delight.

With vast neural networks they're endowed,
To learn and comprehend with ease,
Their training on vast texts bestowed,
To grasp language's intricate keys.

They can generate, translate, and more,
A tool for writers, a scholar's delight,
A marvel of our digital core,
A beacon of AI's shining light.

So, large language models, we now know,
Are wonders of our time, to ponder and show.


## Predict

In [18]:
predict = dspy.Predict('question -> answer')
predict(question = 'what is the capital of France?')

Prediction(
    answer='The capital of France is Paris.'
)

In [19]:
class QA(dspy.Signature):
    question = dspy.InputField()
    answer = dspy.OutputField()

predict = dspy.Predict(QA)
predict(question = 'what is the capital of France?')

Prediction(
    answer='The capital of France is Paris.'
)

### Classification

In [20]:
from typing import Literal

class Classify(dspy.Signature):
    """
    Classify sentiment of a given sentence.
    """

    sentence: str = dspy.InputField()
    sentiment: Literal['positive', 'negative', 'neutral'] = dspy.OutputField()
    confidence: float = dspy.OutputField()

classify = dspy.Predict(Classify)

classify(sentence="This book was super fun to read, though not the last chapter.")

Prediction(
    sentiment='positive',
    confidence=0.95
)

In [21]:
classify(sentence="it was not that good only watched for bill murray")

Prediction(
    sentiment='negative',
    confidence=0.85
)

### Information Extraction

In [22]:
class ExtractInfo(dspy.Signature):
    """Extract structured information from text."""

    text: str = dspy.InputField()
    title: str = dspy.OutputField()
    headings: list[str] = dspy.OutputField()
    entities: list[dict[str, str]] = dspy.OutputField(desc="a list of entities and their metadata")

module = dspy.Predict(ExtractInfo)

text = "Apple Inc. announced its latest iPhone 14 today." \
    "The CEO, Tim Cook, highlighted its new features in a press release."

response = module(text=text)
response

Prediction(
    title='Apple Inc. Announces iPhone 14 with New Features',
    headings=['Company', 'Product', 'CEO', 'Press Release'],
    entities=[{'type': 'company', 'name': 'Apple Inc.'}, {'type': 'product', 'name': 'iPhone 14'}, {'type': 'person', 'name': 'Tim Cook'}, {'type': 'document', 'name': 'Press Release'}]
)

## Chain of Thought

In [24]:
math = dspy.ChainOfThought("question -> answer: float")
math(question="Two dice are tossed. What is the probability that the sum equals two?")

Prediction(
    reasoning="To find the probability that the sum of two dice equals two, we need to consider the possible outcomes of rolling two dice and the number of outcomes where the sum is two.\n\nThere are 6 sides on each die, so there are a total of 6 * 6 = 36 possible outcomes when rolling two dice.\n\nNow, let's find the outcomes where the sum is two. The only way to get a sum of two is by rolling a 1 on the first die and a 1 on the second die. There are 6 sides on each die, so the probability of rolling a 1 on each die is 1/6.\n\nSo, the probability of the sum of two dice equaling two is (1/6) * (1/6) = 1/36.",
    answer=0.02777777777777778
)

In [25]:
(1/6) ** 2

0.027777777777777776

In [26]:
math = dspy.ChainOfThought("question -> answer: int")
math(question="What is the age of the USA as of 2025?")

Prediction(
    reasoning='To calculate the age of the USA as of 2025, we need to know the year the USA was founded, which is 1776. Then, we subtract the year of founding from the target year (2025) to get the age.',
    answer=1779
)

In [27]:
2025-1776

249

In [28]:
math(question="we got 1000 transaction last year. it increases 5% each year in 10 years, how many transactions should we have?")

Prediction(
    reasoning='We need to calculate the total number of transactions after a 10% annual increase over 10 years.',
    answer=1593
)

In [29]:
initial = 1000
for i in range(10):
    initial += initial * 0.05

print(initial)

1628.894626777441


In [53]:
class AnomalyDetection(dspy.Signature):
    data = dspy.InputField()

    date = dspy.OutputField(desc='The date the anomaly is detected in value. If no anomaly found, say None')

anomaly_detection = dspy.ChainOfThought(AnomalyDetection)

data = """
date         value
2024-02-01   10
2024-02-02   12
2024-02-03   11
2024-02-04   20
2024-02-05   13
2024-02-06   14
2024-02-07   13
2024-02-08   14
"""

anomaly_detection(data=data)

Prediction(
    reasoning='An anomaly is detected on 2024-02-04 with a value of 20, which is significantly higher than the previous values.',
    date='2024-02-04'
)

## Program of Thought

In [30]:
pot = dspy.ProgramOfThought('question -> answer')
pot(question="What is the age of the USA as of 2025?")

Age of the USA as of 2025: 249


Prediction(
    reasoning='The final generated code calculates the age of the USA by subtracting the year of founding (1776) from the year 2025. The result is then printed as the age of the USA as of 2025.',
    answer='The age of the USA as of 2025 is 249 years.'
)

In [31]:
lm.inspect_history()





[2025-03-16T02:03:07.598290]

System message:

Your input fields are:
1. `question` (str)
2. `final_generated_code` (str): python code that answers the question
3. `code_output` (str): output of previously-generated python code

Your output fields are:
1. `reasoning` (str)
2. `answer` (str)

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## final_generated_code ## ]]
{final_generated_code}

[[ ## code_output ## ]]
{code_output}

[[ ## reasoning ## ]]
{reasoning}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Given the final code `question`, `final_generated_code`, `code_output`, provide the final `answer`.


User message:

[[ ## question ## ]]
What is the age of the USA as of 2025?

[[ ## final_generated_code ## ]]
year_of_founding = 1776
year_of_founding_to_2025 = 2025 - year_of_founding
age_of_usa = year_of_founding_to_2025

print

In [32]:
pot(question='we got 1000 transaction last year. it increases 5% each year in 10 years, how many transactions should we have?')

Prediction(
    reasoning='The final generated code calculates the total number of transactions after 10 years, assuming a 5% increase each year. The formula used is (1 + 0.05) ^ 10 * 1000, where 1000 is the initial number of transactions, 0.05 is the annual increase rate, and (1 + 0.05) ^ 10 calculates the total increase after 10 years.',
    answer='1628.89'
)

In [33]:
lm.inspect_history()





[2025-03-16T02:03:14.223329]

System message:

Your input fields are:
1. `question` (str)
2. `final_generated_code` (str): python code that answers the question
3. `code_output` (str): output of previously-generated python code

Your output fields are:
1. `reasoning` (str)
2. `answer` (str)

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## final_generated_code ## ]]
{final_generated_code}

[[ ## code_output ## ]]
{code_output}

[[ ## reasoning ## ]]
{reasoning}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Given the final code `question`, `final_generated_code`, `code_output`, provide the final `answer`.


User message:

[[ ## question ## ]]
we got 1000 transaction last year. it increases 5% each year in 10 years, how many transactions should we have?

[[ ## final_generated_code ## ]]
total_transactions = 1000 * (1 + 0.05) ** 10

